In [16]:
# Python
import pandas as pd
import numpy as np
#
from pathlib import Path
#
import xgboost
#
import importlib
import utilities.train_test as train_test
import utilities.comparison as comparison

In [17]:
df = pd.read_csv('../../../data/df_monthly_returns_complete_percentage.csv', index_col='Date')
df_overview = pd.read_csv('../../../data/df_overview.csv', index_col=0)

min_date = pd.to_datetime(df.index.max()) - pd.DateOffset(months=12*15)
min_datestr = min_date.strftime('%Y-%m-%d')

#df = df.loc[df.index >= min_datestr]

In [18]:
importlib.reload(train_test)
 
df = pd.read_csv('../../../data/df_monthly_returns_complete_percentage.csv', index_col='Date')

file=Path("../../../data/df_tabular_multi.csv")

if file.exists():
    df_tabular_multi=pd.read_csv(file, index_col=0)
else:
    importlib.reload(train_test)
    df_tabular_multi = train_test.get_dataframe_tabular_multi(df)
    df_tabular_multi.to_csv(file)

## Direct forecasting

In [19]:
# True values
y_true = df.mean(axis=1)

## 1 Month

In [20]:
df_tabular_multi

,month,year,date,m_return(t-11),m_return(t-10),m_return(t-9),m_return(t-8),m_return(t-7),m_return(t-6),m_return(t-5),...,industry_Traders & Distributors,industry_Transportation,industry_Transportation Infrastructure,industry_Utilities,stock_exchange_ETR,stock_exchange_LON,stock_exchange_NAS,stock_exchange_NYS,stock_exchange_TKS,trailing_pe_scaled
0,10,2000,2000-10-01,1.13,1.11,1.02,1.00,1.00,1.00,1.00,...,0,0,0,0,0,1,0,0,0,0.001718
1,11,2000,2000-11-01,1.11,1.02,1.00,1.00,1.00,1.00,1.00,...,0,0,0,0,0,1,0,0,0,0.001718
2,12,2000,2000-12-01,1.02,1.00,1.00,1.00,1.00,1.00,1.01,...,0,0,0,0,0,1,0,0,0,0.001718
3,1,2001,2001-01-01,1.00,1.00,1.00,1.00,1.00,1.01,1.00,...,0,0,0,0,0,1,0,0,0,0.001718
4,2,2001,2001-02-01,1.00,1.00,1.00,1.00,1.01,1.00,1.00,...,0,0,0,0,0,1,0,0,0,0.001718
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
474406,4,2024,2024-04-01,0.98,1.07,1.13,0.95,0.93,1.02,1.09,...,0,1,0,0,0,0,0,1,0,0.002044
474407,5,2024,2024-05-01,1.07,1.13,0.95,0.93,1.02,1.09,1.09,...,0,1,0,0,0,0,0,1,0,0.002044
474408,6,2024,2024-06-01,1.13,0.95,0.93,1.02,1.09,1.09,1.00,...,0,1,0,0,0,0,0,1,0,0.002044
474409,7,2024,2024-07-01,0.95,0.93,1.02,1.09,1.09,1.00,1.04,...,0,1,0,0,0,0,0,1,0,0.002044


In [21]:
importlib.reload(train_test)

X_train, y_train, X_test, y_test = train_test.get_train_test(df_tabular_multi, months=60)

model = xgboost.XGBRegressor(n_estimators=1000)
model.fit(X_train, y_train,
          eval_set=[(X_train, y_train), (X_test, y_test)],
          verbose=False)

2019-08-01


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=1000, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [22]:
X_train

,month,year,m_return(t-11),m_return(t-10),m_return(t-9),m_return(t-8),m_return(t-7),m_return(t-6),m_return(t-5),m_return(t-4),...,industry_Traders & Distributors,industry_Transportation,industry_Transportation Infrastructure,industry_Utilities,stock_exchange_ETR,stock_exchange_LON,stock_exchange_NAS,stock_exchange_NYS,stock_exchange_TKS,trailing_pe_scaled
0,10,2000,1.13,1.11,1.02,1.00,1.00,1.00,1.00,1.00,...,0,0,0,0,0,1,0,0,0,0.001718
1,11,2000,1.11,1.02,1.00,1.00,1.00,1.00,1.00,1.01,...,0,0,0,0,0,1,0,0,0,0.001718
2,12,2000,1.02,1.00,1.00,1.00,1.00,1.00,1.01,1.00,...,0,0,0,0,0,1,0,0,0,0.001718
3,1,2001,1.00,1.00,1.00,1.00,1.00,1.01,1.00,1.00,...,0,0,0,0,0,1,0,0,0,0.001718
4,2,2001,1.00,1.00,1.00,1.00,1.01,1.00,1.00,1.00,...,0,0,0,0,0,1,0,0,0,0.001718
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
474346,4,2019,1.07,1.00,1.06,1.00,1.09,0.90,1.05,0.90,...,0,1,0,0,0,0,0,1,0,0.002044
474347,5,2019,1.00,1.06,1.00,1.09,0.90,1.05,0.90,1.15,...,0,1,0,0,0,0,0,1,0,0.002044
474348,6,2019,1.06,1.00,1.09,0.90,1.05,0.90,1.15,1.05,...,0,1,0,0,0,0,0,1,0,0.002044
474349,7,2019,1.00,1.09,0.90,1.05,0.90,1.15,1.05,1.00,...,0,1,0,0,0,0,0,1,0,0.002044


In [23]:
importlib.reload(comparison)
months_1m = 1
X_train_1m, y_train_1m, X_test_1m, y_test_1m = train_test.get_train_test(df_tabular_multi, months=1)
# reset index
X_train_1m.reset_index(drop=True, inplace=True)
y_train_1m.reset_index(drop=True, inplace=True)
X_test_1m.reset_index(drop=True, inplace=True)
y_test_1m.reset_index(drop=True, inplace=True)

2024-07-01


In [24]:
# Predictions
y_train_pred_1m = model.predict(X_train_1m)
y_test_pred_1m = model.predict(X_test_1m)

#### Train data

In [25]:
importlib.reload(comparison)
y_train_mean_pred_1m, y_test_mean_pred_1m = comparison.get_train_test_mean_pred(y_train_pred_1m, y_test_pred_1m, len(df.columns))

### 1 month Actual vs Prediction

In [26]:
importlib.reload(comparison)
comparison.generate_plot(df, df_tabular_multi, y_train_mean_pred_1m, y_test_mean_pred_1m)

#### Overview table

In [27]:
importlib.reload(comparison)

# Get train true values followed with predicted month/s
y_train_1m_list = y_train_1m['m_return_target(t+1)'].tolist()
y_test_1m_list = y_test_pred_1m.tolist()
#
df_to_evaluate_1m = comparison.get_df_from_pred_list(df, y_train_1m_list, y_test_1m_list)
df_to_evaluate_1m = df_to_evaluate_1m - 1
df_to_evaluate_1m

,RS1.L,KE,TEG.DE,LEG.DE,SCS,HNI,AVT,ACCO,VNA.DE,7912.T,...,DEQ.DE,KIDS,HALO,MATW,9842.T,KVHI,MOON.L,NEO,6055.T,UNP
0,0.000000,0.00000,-0.100000,0.000000,-0.230000,0.010000,-0.340000,0.000000,0.00000,-0.060000,...,0.000000,0.000000,0.000000,0.070000,0.000000,-0.340000,0.00000,0.00000,0.000000,-0.010000
1,0.000000,0.00000,0.000000,-0.060000,0.030000,0.050000,0.220000,0.000000,-0.04000,0.050000,...,0.000000,0.000000,0.000000,0.090000,0.000000,-0.160000,0.00000,0.00000,0.000000,0.090000
2,0.010000,0.00000,0.550000,0.040000,0.090000,-0.020000,0.280000,-0.080000,0.02000,-0.060000,...,0.000000,0.290000,-0.090000,-0.020000,-0.040000,0.690000,-0.14000,-0.24000,0.000000,0.050000
3,0.000000,-0.03000,0.090000,-0.130000,-0.050000,-0.010000,-0.110000,-0.020000,-0.10000,-0.080000,...,-0.480000,0.000000,0.000000,0.010000,0.000000,-0.260000,0.00000,0.00000,0.000000,0.040000
4,0.000000,0.00000,-0.060000,0.000000,-0.150000,-0.070000,-0.160000,0.000000,0.00000,0.020000,...,1.180000,0.000000,0.000000,0.050000,0.000000,0.150000,0.00000,0.00000,0.000000,0.020000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
282,-0.050000,0.10000,0.060000,0.020000,0.140000,0.120000,0.120000,0.060000,0.06000,0.060000,...,0.090000,0.070000,0.160000,0.050000,-0.010000,0.080000,0.03000,-0.02000,-0.140000,-0.020000
283,0.000000,-0.04000,-0.040000,-0.030000,-0.050000,-0.040000,-0.060000,-0.060000,-0.05000,0.110000,...,0.070000,-0.090000,0.180000,-0.110000,0.010000,-0.100000,0.19000,0.01000,0.140000,-0.020000
284,0.190000,0.08000,0.020000,0.060000,0.120000,0.220000,0.050000,0.090000,0.07000,-0.080000,...,0.100000,0.070000,0.060000,0.160000,-0.060000,-0.040000,0.14000,0.28000,-0.100000,0.090000
285,-0.040000,-0.22000,0.060000,0.080000,-0.020000,-0.020000,0.030000,0.070000,0.10000,0.060000,...,-0.050000,0.040000,0.160000,-0.130000,-0.040000,0.000000,-0.03000,-0.07000,-0.110000,0.040000


In [28]:
importlib.reload(comparison)
df_pred_1m, raw_weights_1m, mu_1m, S_1m, sigma_1m, sharpe_1m = comparison.get_portfolio_performance(df_to_evaluate_1m,
                                                                                                    "xgbooster_weights_1m.csv",
                                                                                                    min_avg_return=-0.5,
                                                                                                    months=12)

allocation, leftover = comparison.create_discrete_allocation(df_pred_1m, raw_weights_1m)

# Create overview
df_view_1m = pd.DataFrame.from_dict(raw_weights_1m, orient='index', columns=['max_sharpe_weight'])
# Extract volatilities (square root of diagonal elements)
df_view_1m['avg_annual_volatility'] = pd.Series(np.sqrt(np.diag(S_1m)), index=S_1m.columns).values
# Set annual returns
df_view_1m['avg_annual_return'] = mu_1m.values
#
df_view_1m

Expected annual return: 80.4%
Annual volatility: 54.6%
Sharpe Ratio: 1.44
Discrete allocation: {'CDW': 37924, '2379.T': 45, '6754.T': 32, 'ASGN': 381, 'GROW.L': 3, 'KFY': 982, 'KFRC': 1178, '7893.T': 46927, 'BMI': 461, 'EBOX.L': 107, 'NBS.L': 74, '7313.T': 63, '6098.T': 218, 'WPP.L': 50631, 'DNLM.L': 885, 'TEP.L': 434, 'ESRT': 911, 'ZBRA': 681, 'NYT': 293, 'SNWS.L': 201, 'INCH.L': 26, 'LAD': 82, '2146.T': 36, '8057.T': 73, '8151.T': 742, '7868.T': 86, 'CDP': 470, 'GOLF': 156, '7272.T': 23, 'RCH.L': 250, '7701.T': 1340, '2154.T': 16, 'NSIT': 507, 'CDNS': 138, 'VNT': 70, '7976.T': 59, 'WSM': 1054, 'FOXT.L': 205, '2124.T': 24, '4641.T': 41, 'LEA': 10340, '7762.T': 42, 'ANIK': 469, '5108.T': 25, '3107.T': 50, 'BBSI': 252, '8084.T': 570, '6701.T': 122, '4732.T': 36, 'JDG.L': 56, '6702.T': 65, '1925.T': 32, 'IVT': 16, '6448.T': 23, 'ESCA': 50, 'BKG.L': 132, '3462.T': 37, '6981.T': 29, '6436.T': 51407, 'HST': 346, '8035.T': 62, '7734.T': 122, 'UTG.L': 167, 'HOG': 103, 'UTMD': 403, 'CLMB': 495

,max_sharpe_weight,avg_annual_volatility,avg_annual_return
RS1.L,0.00000,8.979876,-0.285367
KE,0.00000,8.989774,-0.599410
TEG.DE,0.00127,9.001995,-0.123928
LEG.DE,0.00000,9.038371,-0.348568
SCS,0.00000,8.988576,-0.165987
...,...,...,...
KVHI,0.00000,8.980887,-0.474605
MOON.L,0.00137,8.949683,0.431999
NEO,0.00028,9.034262,-0.181364
6055.T,0.00000,8.963966,-0.237748


## 6 Months


In [29]:
importlib.reload(train_test)
X_train, y_train, X_test, y_test = train_test.get_train_test(df_tabular_multi, months=60, target_key='m_return_target(t+6)')

# 
model = xgboost.XGBRegressor(n_estimators=1000)
model.fit(X_train, y_train,
          eval_set=[(X_train, y_train), (X_test, y_test)],
          verbose=False)

2019-08-01


XGBoostError: [20:16:53] /Users/runner/miniforge3/conda-bld/xgboost-split_1738880277541/work/src/data/data.cc:514: Check failed: valid: Label contains NaN, infinity or a value too large.
Stack trace:
  [bt] (0) 1   libxgboost.dylib                    0x000000013a0c4170 dmlc::LogMessageFatal::~LogMessageFatal() + 124
  [bt] (1) 2   libxgboost.dylib                    0x000000013a218778 xgboost::MetaInfo::SetInfoFromHost(xgboost::Context const&, xgboost::StringView, xgboost::Json) + 2804
  [bt] (2) 3   libxgboost.dylib                    0x000000013a217b6c xgboost::MetaInfo::SetInfo(xgboost::Context const&, xgboost::StringView, xgboost::StringView) + 628
  [bt] (3) 4   libxgboost.dylib                    0x000000013a0dad84 XGDMatrixSetInfoFromInterface + 244
  [bt] (4) 5   libffi.8.dylib                      0x000000010132004c ffi_call_SYSV + 76
  [bt] (5) 6   libffi.8.dylib                      0x000000010131d834 ffi_call_int + 1404
  [bt] (6) 7   _ctypes.cpython-311-darwin.so       0x0

In [73]:
importlib.reload(comparison)
months_6m = 6
X_train_6m, y_train_6m, X_test_6m, y_test_6m = train_test.get_train_test(df_tabular_multi, months=6,
                                                                         target_key='m_return_target(t+6)')

# Predictions
y_train_pred_6m = model.predict(X_train_6m)
y_test_pred_6m = model.predict(X_test_6m)

1 (474411, 85)
2 (466146, 85)


#### Train data

In [71]:
importlib.reload(comparison)
y_train_mean_pred_6m, y_test_mean_pred_6m = comparison.get_train_test_mean_pred(y_train_pred_6m, y_test_pred_6m, len(df.columns))

### 6 Months Actual vs Prediction

In [1]:
y_test_mean_pred_6m

NameError: name 'y_test_mean_pred_6m' is not defined

In [59]:
importlib.reload(comparison)
comparison.generate_plot(df, df_tabular_multi, y_train_mean_pred_6m, y_test_mean_pred_6m)

#### Overview

In [60]:
# Get train true values followed with predicted month/s
y_train_6m_list = y_train_6m['m_return_target(t+6)'].tolist()
y_test_6m_list = y_test_pred_6m.tolist()
#
df_to_evaluate_6m = comparison.get_df_from_pred_list(df, y_train_1m_list, y_test_1m_list)
df_to_evaluate_6m = df_to_evaluate_6m - 1
df_to_evaluate_6m

,RS1.L,KE,TEG.DE,LEG.DE,SCS,HNI,AVT,ACCO,VNA.DE,7912.T,...,DEQ.DE,KIDS,HALO,MATW,9842.T,KVHI,MOON.L,NEO,6055.T,UNP
0,0.000000,0.00000,-0.100000,0.000000,-0.230000,0.010000,-0.340000,0.000000,0.00000,-0.060000,...,0.000000,0.000000,0.000000,0.070000,0.000000,-0.340000,0.00000,0.00000,0.000000,-0.010000
1,0.000000,0.00000,0.000000,-0.060000,0.030000,0.050000,0.220000,0.000000,-0.04000,0.050000,...,0.000000,0.000000,0.000000,0.090000,0.000000,-0.160000,0.00000,0.00000,0.000000,0.090000
2,0.010000,0.00000,0.550000,0.040000,0.090000,-0.020000,0.280000,-0.080000,0.02000,-0.060000,...,0.000000,0.290000,-0.090000,-0.020000,-0.040000,0.690000,-0.14000,-0.24000,0.000000,0.050000
3,0.000000,-0.03000,0.090000,-0.130000,-0.050000,-0.010000,-0.110000,-0.020000,-0.10000,-0.080000,...,-0.480000,0.000000,0.000000,0.010000,0.000000,-0.260000,0.00000,0.00000,0.000000,0.040000
4,0.000000,0.00000,-0.060000,0.000000,-0.150000,-0.070000,-0.160000,0.000000,0.00000,0.020000,...,1.180000,0.000000,0.000000,0.050000,0.000000,0.150000,0.00000,0.00000,0.000000,0.020000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
282,-0.050000,0.10000,0.060000,0.020000,0.140000,0.120000,0.120000,0.060000,0.06000,0.060000,...,0.090000,0.070000,0.160000,0.050000,-0.010000,0.080000,0.03000,-0.02000,-0.140000,-0.020000
283,0.000000,-0.04000,-0.040000,-0.030000,-0.050000,-0.040000,-0.060000,-0.060000,-0.05000,0.110000,...,0.070000,-0.090000,0.180000,-0.110000,0.010000,-0.100000,0.19000,0.01000,0.140000,-0.020000
284,0.190000,0.08000,0.020000,0.060000,0.120000,0.220000,0.050000,0.090000,0.07000,-0.080000,...,0.100000,0.070000,0.060000,0.160000,-0.060000,-0.040000,0.14000,0.28000,-0.100000,0.090000
285,-0.040000,-0.22000,0.060000,0.080000,-0.020000,-0.020000,0.030000,0.070000,0.10000,0.060000,...,-0.050000,0.040000,0.160000,-0.130000,-0.040000,0.000000,-0.03000,-0.07000,-0.110000,0.040000


In [61]:
importlib.reload(comparison)
df_pred_6m, raw_weights_6m, mu_6m, S_6m, sigma_6m, sharpe_6m = comparison.get_portfolio_performance(df_to_evaluate_6m,
                                                                                                    "xgbooster_weights_6m.csv",
                                                                                                    min_avg_return=-0.5,
                                                                                                    months=12)
comparison.create_discrete_allocation(df_pred_6m, raw_weights_6m)

# Create overview
df_view_6m = pd.DataFrame.from_dict(raw_weights_6m, orient='index', columns=['max_sharpe_weight'])
# Extract volatilities (square root of diagonal elements)
df_view_6m['avg_annual_volatility'] = pd.Series(np.sqrt(np.diag(S_6m)), index=S_6m.columns).values
# Set annual returns
df_view_6m['avg_annual_return'] = mu_6m.values
#
df_view_6m

Expected annual return: 80.4%
Annual volatility: 54.6%
Sharpe Ratio: 1.44
Discrete allocation: {'CDW': 37924, '2379.T': 45, '6754.T': 32, 'ASGN': 381, 'GROW.L': 3, 'KFY': 982, 'KFRC': 1178, '7893.T': 46927, 'BMI': 461, 'EBOX.L': 107, 'NBS.L': 74, '7313.T': 63, '6098.T': 218, 'WPP.L': 50631, 'DNLM.L': 885, 'TEP.L': 434, 'ESRT': 911, 'ZBRA': 681, 'NYT': 293, 'SNWS.L': 201, 'INCH.L': 26, 'LAD': 82, '2146.T': 36, '8057.T': 73, '8151.T': 742, '7868.T': 86, 'CDP': 470, 'GOLF': 156, '7272.T': 23, 'RCH.L': 250, '7701.T': 1340, '2154.T': 16, 'NSIT': 507, 'CDNS': 138, 'VNT': 70, '7976.T': 59, 'WSM': 1054, 'FOXT.L': 205, '2124.T': 24, '4641.T': 41, 'LEA': 10340, '7762.T': 42, 'ANIK': 469, '5108.T': 25, '3107.T': 50, 'BBSI': 252, '8084.T': 570, '6701.T': 122, '4732.T': 36, 'JDG.L': 56, '6702.T': 65, '1925.T': 32, 'IVT': 16, '6448.T': 23, 'ESCA': 50, 'BKG.L': 132, '3462.T': 37, '6981.T': 29, '6436.T': 51407, 'HST': 346, '8035.T': 62, '7734.T': 122, 'UTG.L': 167, 'HOG': 103, 'UTMD': 403, 'CLMB': 495

,max_sharpe_weight,avg_annual_volatility,avg_annual_return
RS1.L,0.00000,8.979876,-0.285367
KE,0.00000,8.989774,-0.599410
TEG.DE,0.00127,9.001995,-0.123928
LEG.DE,0.00000,9.038371,-0.348568
SCS,0.00000,8.988576,-0.165987
...,...,...,...
KVHI,0.00000,8.980887,-0.474605
MOON.L,0.00137,8.949683,0.431999
NEO,0.00028,9.034262,-0.181364
6055.T,0.00000,8.963966,-0.237748


## 12 Months

In [62]:
importlib.reload(train_test)
X_train, y_train, X_test, y_test = train_test.get_train_test(df_tabular_multi, months=60, target_key='m_return_target(t+12)')

# 
model = xgboost.XGBRegressor(n_estimators=1000)
model.fit(X_train, y_train,
          eval_set=[(X_train, y_train), (X_test, y_test)],
          verbose=False)

1 (474411, 85)
2 (456228, 85)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=1000, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [63]:
len(X_test)

80997

In [64]:
months_12m = 12
X_train_12m, y_train_12m, X_test_12m, y_test_12m = train_test.get_train_test(df_tabular_multi, months=12,
                                                                             target_key='m_return_target(t+12)')

1 (474411, 85)
2 (456228, 85)


In [65]:
# Predictions
y_train_pred_12m = model.predict(X_train_12m)
y_test_pred_12m = model.predict(X_test_12m)

#### Train-data

In [66]:
importlib.reload(comparison)
y_train_mean_pred_12m, y_test_mean_pred_12m = comparison.get_train_test_mean_pred(y_train_pred_12m, y_test_pred_12m, len(df.columns))

### 12 Months Actual vs Prediction

In [67]:
importlib.reload(comparison)
comparison.generate_plot(df, df_tabular_multi, y_train_mean_pred_12m, y_test_mean_pred_12m)

#### Overview table

In [68]:
importlib.reload(comparison)

# Get train true values followed with predicted month/s
y_train_12m_list = y_train_12m['m_return_target(t+12)'].tolist()
y_test_12m_list = y_test_pred_12m.tolist()
#
df_to_evaluate_12m = comparison.get_df_from_pred_list(df, y_train_12m_list, y_test_12m_list)
df_to_evaluate_12m = df_to_evaluate_12m - 1
df_to_evaluate_12m

,RS1.L,KE,TEG.DE,LEG.DE,SCS,HNI,AVT,ACCO,VNA.DE,7912.T,...,DEQ.DE,KIDS,HALO,MATW,9842.T,KVHI,MOON.L,NEO,6055.T,UNP
0,0.000000,0.000000,0.320000,0.000000,0.050000,0.100000,0.140000,0.000000,0.000000,0.140000,...,0.000000,0.000000,0.000000,0.040000,0.000000,0.530000,0.000000,0.000000,0.010000,0.110000
1,0.000000,0.000000,0.050000,-0.020000,0.090000,0.060000,0.150000,1.430000,-0.020000,0.010000,...,0.000000,0.000000,0.000000,0.050000,0.000000,-0.170000,0.000000,0.000000,0.000000,0.060000
2,0.000000,0.000000,-0.150000,0.000000,0.040000,0.090000,0.070000,0.000000,0.000000,-0.010000,...,-0.120000,0.000000,0.000000,0.020000,0.000000,-0.010000,0.000000,0.000000,0.000000,0.040000
3,0.010000,0.000000,0.040000,0.000000,0.070000,0.010000,0.050000,-0.650000,0.000000,-0.100000,...,0.000000,0.000000,0.000000,0.010000,0.000000,0.230000,0.000000,-0.050000,-0.010000,0.090000
4,0.000000,0.000000,-0.020000,0.000000,-0.030000,-0.030000,-0.010000,0.060000,0.000000,0.100000,...,0.010000,-0.100000,0.000000,-0.030000,0.000000,0.020000,0.000000,0.000000,0.010000,-0.020000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,-0.050000,0.100000,0.060000,0.020000,0.140000,0.120000,0.120000,0.060000,0.060000,0.060000,...,0.090000,0.070000,0.160000,0.050000,-0.010000,0.080000,0.030000,-0.020000,-0.140000,-0.020000
272,0.000000,-0.040000,-0.040000,-0.030000,-0.050000,-0.040000,-0.060000,-0.060000,-0.050000,0.110000,...,0.070000,-0.090000,0.180000,-0.110000,0.010000,-0.100000,0.190000,0.010000,0.140000,-0.020000
273,0.190000,0.080000,0.020000,0.060000,0.120000,0.220000,0.050000,0.090000,0.070000,-0.080000,...,0.100000,0.070000,0.060000,0.160000,-0.060000,-0.040000,0.140000,0.280000,-0.100000,0.090000
274,-0.040000,-0.220000,0.060000,0.080000,-0.020000,-0.020000,0.030000,0.070000,0.100000,0.060000,...,-0.050000,0.040000,0.160000,-0.130000,-0.040000,0.000000,-0.030000,-0.070000,-0.110000,0.040000


In [69]:
importlib.reload(comparison)
df_pred_12m, raw_weights_12m, mu_12m, S_12m, sigma_12m, sharpe_12m = comparison.get_portfolio_performance(df_to_evaluate_12m,
                                                                                                          "xgbooster_weights_12m.csv",
                                                                                                          min_avg_return=-0.5,
                                                                                                          months=12)
comparison.create_discrete_allocation(df_pred_12m, raw_weights_12m)

# Create overview
df_view_12m = pd.DataFrame.from_dict(raw_weights_12m, orient='index', columns=['max_sharpe_weight'])
# Extract volatilities (square root of diagonal elements)
df_view_12m['avg_annual_volatility'] = pd.Series(np.sqrt(np.diag(S_12m)), index=S_12m.columns).values
# Set annual returns
df_view_12m['avg_annual_return'] = mu_12m.values
# 
df_view_12m['return_last_period(12m)'] = round(df.tail(12).prod() - 1, 2)
df_view_12m

Expected annual return: 67.5%
Annual volatility: 52.0%
Sharpe Ratio: 1.26
Discrete allocation: {'TEG.DE': 596, 'LEG.DE': 550, 'VNA.DE': 626, 'SGRO.L': 699, 'CBRE': 1355, 'PSON.L': 23027, 'BBOX.L': 739, 'REL.L': 926, 'GPE.L': 566, 'HAS': 22760, 'JLL': 48072, 'LAND.L': 503, 'OLED': 1091, 'RYN': 19376, '2379.T': 24357, 'BYG.L': 777, 'DB1.DE': 788, 'AVB': 1190, 'BLND.L': 572, '8954.T': 534, 'ACN': 659, 'SVS.L': 25148, 'AGNC': 660, 'EBOX.L': 16774, '7994.T': 658, '3283.T': 446, 'NBS.L': 1222, '6183.T': 534, '7984.T': 728, 'CCK': 24208, 'UU.L': 664, 'INF.L': 1174, 'HMSO.L': 379, 'WPP.L': 738, 'WKP.L': 616, '6028.T': 1187, 'NLY': 1147, '7951.T': 1173, '7846.T': 508, 'SBAC': 445, 'ELME': 2621, 'INS.DE': 802, 'AT1.DE': 270, 'EXPN.L': 20800, 'GRI.L': 24370, 'CUZ': 1368, 'PHP.L': 610, 'NL': 1049, 'BALL': 1181, 'DHR': 658, 'KIM': 17463, 'ECOR.L': 661, '8151.T': 794, 'PLD': 534, 'RHI': 1174, '7868.T': 582, 'HEAD.L': 583, 'RCH.L': 701, 'GWI.L': 31162, 'ZWS': 1174, '2154.T': 533, 'ADV.DE': 1117, 'CAH

,max_sharpe_weight,avg_annual_volatility,avg_annual_return,return_last_period(12m)
RS1.L,0.00000,8.622440,0.078503,0.08
KE,0.00000,8.627400,-0.352755,-0.35
TEG.DE,0.00389,8.626091,0.629808,0.63
LEG.DE,0.00221,8.632463,0.494099,0.49
SCS,0.00000,8.626812,0.216904,0.22
...,...,...,...,...
KVHI,0.00000,8.615108,-0.116586,-0.12
MOON.L,0.00079,8.624348,0.222090,0.22
NEO,0.00000,8.665323,0.282414,0.28
6055.T,0.00000,8.640165,-0.264113,-0.26
